In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import json
import numpy as np
import pandas as pd
from cilin import Cilin
from CompoTree import Radicals
from collections import Counter
from itertools import product, chain
from random import sample
from utils import *

C = Cilin(trad=True)
all_words = chain.from_iterable(C.category_split().values())
Tagger = RadicalSemanticTagger(all_words=all_words)
DTM = DocumentTermMatrix(RSTagger=Tagger, Cilin=C, level=3)

In [15]:
from sklearn.naive_bayes import MultinomialNB

df_feat, df_tgt = DTM.pd, DTM.df_tgt
X_train, X_test, y_train, y_test = train_test_split(
    df_feat, df_tgt, tgt_col="lev1", test_size = 0.3, random_state = 20
)

model = MultinomialNB(alpha=0.001)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print("Not predicted:", sorted(set(y_train) - set(predictions)))
classification_report(y_test, predictions)

Not predicted: ['F 動作', 'J 關聯', 'L 敬語']


/home/liao/py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/liao/py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/liao/py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
A 人,0.739130,0.472222,0.576271,36.000000
B 物,0.536842,0.708333,0.610778,72.000000
C 時空,0.666667,0.111111,0.190476,18.000000
D 抽象事物,0.307692,0.230769,0.263736,52.000000
E 特徵,0.400000,0.629630,0.489209,54.000000
F 動作,0.000000,0.000000,0.000000,17.000000
G 心理活動,0.454545,0.384615,0.416667,13.000000
H 活動,0.414013,0.730337,0.528455,89.000000
I 現象與狀態,0.000000,0.000000,0.000000,39.000000
J 關聯,0.000000,0.000000,0.000000,13.000000


In [16]:
np.exp(model.feature_log_prob_)

array([[4.11598565e-02, 4.96843623e-02, 4.41220427e-02, ...,
        2.79253631e-06, 2.79253631e-06, 2.79253631e-06],
       [2.24052083e-02, 1.20062276e-02, 1.23935791e-02, ...,
        1.41877072e-03, 3.30589601e-03, 1.10385101e-03],
       [5.52912494e-02, 1.88534058e-02, 9.86696419e-03, ...,
        1.38555735e-04, 5.36659960e-06, 5.36659960e-06],
       ...,
       [4.54475702e-02, 1.87442532e-02, 1.02691607e-02, ...,
        6.77140501e-06, 6.77140501e-06, 6.77140501e-06],
       [5.03904293e-02, 2.34591428e-02, 9.60889724e-03, ...,
        4.63322951e-06, 4.63322951e-06, 4.63322951e-06],
       [3.35862477e-02, 4.62446589e-03, 5.72065037e-05, ...,
        5.72065037e-05, 5.72065037e-05, 5.72065037e-05]])

In [17]:
df_feat

,NULL,人,人倫關係,人體內部,人體四肢,人體性質,人體精神,人體頭部,住宿,動物軀體,...,顏色_性質,顏色_植物,顏色_武器,顏色_無生命,顏色_生命性質,顏色_生活器物,顏色_禮樂,顏色_穿著器物,顏色_野獸,顏色_顏色
0,0.174809,0.382876,0.366798,0.036422,0.131761,0.0,0.069208,0.099295,0.093680,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.074476,0.127659,0.156270,0.000000,0.000000,0.0,0.058971,0.126911,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.138328,0.052691,0.064500,0.000000,0.000000,0.0,0.219058,0.104764,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.197194,0.225341,0.137922,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.345794,0.564503,0.000000,0.000000,0.000000,0.0,0.000000,0.056119,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,0.045808,0.000000,0.000000,0.000000,0.046037,0.0,0.000000,0.052040,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1424,0.000000,0.000000,0.172940,0.000000,0.124246,0.0,0.000000,0.140449,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1425,0.139731,0.079838,0.000000,0.000000,0.421283,0.0,0.000000,0.079370,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1426,0.130533,0.149165,0.000000,0.000000,0.196776,0.0,0.310072,0.222437,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,30), dpi=600)

mat = np.exp(model.feature_log_prob_).T
ax = sns.heatmap(mat)
ax.set_yticks(range(mat.shape[0])) # <--- set the ticks first
ax.set_yticklabels(range(mat.shape[0]))
plt.show()
# ax = sns.heatmap(tf_idf_mat.toarray(), linewidth=0.5)
# plt.imshow(mat, cmap='hot', interpolation='nearest')